# Prova final da disciplina Análise Estatística de Dados e Informações - PPCA

Prof. João Gabriel de Moraes Souza

Aluno: Demétrius de Almeida Jubé

# Análise financeira do portifólio de varejistas da área de vestuário do Brasil

Para a nossa análise, serão consideradas as ações de quatro empresas de vestuário que atuam na B3:

| Empresa | Código | Destaque ou perfil corporativo |
|:---|:---:|---|
| Marisa | AMAR3.SA | A maior rede de moda feminina e lingerie do Brasil. É Prêmio Top of Mind na categoria e possui lojas físicas e online. |
| Guararapes | GUAR3.SA | Dona das marcas Riachuelo, Carter's e Midway Financeira. Toda  a confecção da Guararapes abastece as mais de 300 lojas da Riachuelo. |
| Arezzo | ARZZ3.SA | Dona das marcas Arezzo, Schutz e AnaCapri. Também vende produtos da marca Vans. É líder no setor de calçados, bolsas e acessórios femininos. Possui e-commerce e lojas físicas. |
| Lojas Renner | LREN3.SA | Dona das Lojas Renner, Camicado, Realize e Youcom. Possui mais de 600 lojas físicas no Brasil, Uruguai e Argentina, além de e-commerce. |

O índice que será utilizado para comparação será o IBOVESPA (^BVSP), e os objetivos da análise são os seguintes:

* Fazer a análise descritiva dos dados das ações
* Encontrar o portfólio que tenha o melhor Índice de Sharpe, juntamente com a Fronteira Eficiente
* Realizar ANOVA e Testes de Hipóteses
* Executar uma Regressão Linear e encontrar a relação de uma ação específica com o IBOVESPA
* Utilizar um modelo de previsão de Machine Learning para alguma ação

## Análise da performance da carteira

Para realização dos objetivos, vamos montar o ambiente e recuperar os dados que serão analisados:

### Setup do ambiente

Aqui, recuperaremos as bibliotecas que serão utilizadas para carregar e analisar os dados:

In [1]:
import pandas as pd
from pandas_datareader import data
import numpy as np
import math
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import seaborn as sns
from scipy import stats
from scipy import optimize

pio.renderers.default = 'notebook_connected'

### Construindo a base de dados com as ações selecionadas

Faremos uma lista das ações que queremos verificar na carteira e buscaremos as informações no Yahoo Finance, utilizando a biblioteca do Panda Datareader para isso.

In [2]:
portfolio = ['AMAR3.SA', 'GUAR3.SA', 'ARZZ3.SA', 'LREN3.SA']
indice = '^BVSP'
acoes = portfolio
acoes.append(indice)
print('Ações que serão pesquisadas:')
acoes

Ações que serão pesquisadas:


['AMAR3.SA', 'GUAR3.SA', 'ARZZ3.SA', 'LREN3.SA', '^BVSP']

In [3]:
acoes_df = pd.DataFrame()
for acao in acoes:
    acoes_df[acao] = data.DataReader(acao,
                                     data_source='yahoo', start='2015-01-01')['Close']

O resultado da pesquisa pode ser verificado abaixo:

In [4]:
acoes_df

,AMAR3.SA,GUAR3.SA,ARZZ3.SA,LREN3.SA,^BVSP
Date,,,,,
2015-01-02,13.190909,10.23000,25.850000,12.446208,48512.000000
2015-01-05,13.090909,9.90625,25.040001,12.100207,47517.000000
2015-01-06,12.281818,9.87375,24.549999,12.239591,48001.000000
2015-01-07,12.509090,10.05500,24.799999,12.447848,49463.000000
2015-01-08,12.372727,9.90750,24.680000,12.462606,49943.000000
...,...,...,...,...,...
2022-09-05,2.500000,10.15000,93.309998,26.969999,112203.000000
2022-09-06,2.360000,10.10000,91.660004,26.030001,109764.000000
2022-09-08,2.430000,10.35000,92.250000,26.020000,109916.000000


Por conta da cisão do Pão de Açucar e do Assaí em março de 2021, os dados recuperados estão vindo a partir dessa data.

Será necessário incluir um índice independente na lista, pois a pesquisa coloca como chave de cada registro a data. É possível fazer isso utilizando a função `reset_index` do `DataFrame`. Ao utilizarmos o argumento `inplace=True`, nós garantimos que não há nenhuma perda de dados.

In [5]:
acoes_df.reset_index(inplace=True)
acoes_df

,Date,AMAR3.SA,GUAR3.SA,ARZZ3.SA,LREN3.SA,^BVSP
0,2015-01-02,13.190909,10.23000,25.850000,12.446208,48512.000000
1,2015-01-05,13.090909,9.90625,25.040001,12.100207,47517.000000
2,2015-01-06,12.281818,9.87375,24.549999,12.239591,48001.000000
3,2015-01-07,12.509090,10.05500,24.799999,12.447848,49463.000000
4,2015-01-08,12.372727,9.90750,24.680000,12.462606,49943.000000
...,...,...,...,...,...,...
1908,2022-09-05,2.500000,10.15000,93.309998,26.969999,112203.000000
1909,2022-09-06,2.360000,10.10000,91.660004,26.030001,109764.000000
1910,2022-09-08,2.430000,10.35000,92.250000,26.020000,109916.000000
1911,2022-09-09,2.560000,10.58000,93.150002,26.850000,112300.000000


### Visualização dos Dados

A Biblioteca Ploty, do Python, nos permite visualizar, de forma gráfica, os dados que recuperamos. Vamos ver o histórico de preços das ações no decorrer do tempo:

In [6]:
figuraHistorico = px.line(title = 'Histórico do preço das ações')
for i in acoes_df.columns[1:]:
  figuraHistorico.add_scatter(x = acoes_df["Date"] ,y = acoes_df[i], name = i)
figuraHistorico.show()